### LangChain Agent Setup for YouTube QA Bot
This notebook creates a LangChain agent that can answer questions based on the Eleo German learning video transcript using FAISS and OpenAI.


In [4]:
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA

In [5]:

from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

# Load FAISS vector store
db = FAISS.load_local(
    folder_path="../data/vectorstores/eleo_faiss",
    embeddings=OpenAIEmbeddings(model="text-embedding-3-small"),
    allow_dangerous_deserialization=True
)
retriever = db.as_retriever()

from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-3.5-turbo"),
    chain_type="stuff",
    retriever=retriever
)


###  Wrap QA Chain as a LangChain Tool


In [6]:
def answer_question_only_result(query: str) -> str:
    return qa_chain.invoke({"query": query})["result"]

qa_tool = Tool(
    name="VideoQA",
    func=answer_question_only_result,
    description="Answers questions about the Eleo German learning video"
)


### Create Zero-Shot Agent (No Memory)


In [7]:
# Set up the LLM again
llm = ChatOpenAI(model="gpt-3.5-turbo")

# Create the agent with your tool
agent = initialize_agent(
    tools=[qa_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  # shows reasoning steps
)

/var/folders/m8/k1bt27ks3h3cd9920twfctb40000gn/T/ipykernel_51347/217662044.py:5: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


### Run the Agent for a Sample Question


In [8]:
response = agent.run("How does Eleo recommend practicing German pronunciation?")
print(response)


/var/folders/m8/k1bt27ks3h3cd9920twfctb40000gn/T/ipykernel_51347/3570120120.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run("How does Eleo recommend practicing German pronunciation?")




> Entering new AgentExecutor chain...
I should use the VideoQA tool to get the answer from the Eleo German learning video.
Action: VideoQA
Action Input: "How does Eleo recommend practicing German pronunciation?"
Observation: Eleo recommends practicing German pronunciation by speaking with oneself in the language being learned, having conversations with native speakers or other German speakers if possible, and listening to authentic content like videos or using platforms like FluentU to expand vocabulary. Speaking with native speakers or listening to them speak is considered ideal for practicing pronunciation.
Thought:I now know the final answer
Final Answer: Eleo recommends practicing German pronunciation by speaking with oneself, having conversations with native speakers, and listening to authentic content like videos.

> Finished chain.
Eleo recommends practicing German pronunciation by speaking with oneself, having conversations with native speakers, and listening to authentic con

### Add Memory to the Agent (ConversationBufferMemory)


In [9]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


/var/folders/m8/k1bt27ks3h3cd9920twfctb40000gn/T/ipykernel_51347/4063055825.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


### Create Conversational Agent (With Memory)


In [10]:
from langchain.agents import initialize_agent, AgentType

agent_with_memory = initialize_agent(
    tools=[qa_tool],
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)


### Run a Multi-Turn Conversation with Memory


In [ ]:
# Ask the agent a question
response_1 = agent_with_memory.run("How does Eleo recommend practicing German pronunciation?")
print(response_1)

# Follow-up question
response_2 = agent_with_memory.run("Can you explain that again more simply?")
print(response_2)




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: VideoQA
Action Input: How does Eleo recommend practicing German pronunciation?
Observation: Eleo recommends practicing German pronunciation by speaking to oneself in the language being learned during daily activities. It is also suggested to engage in conversations with native speakers or other German speakers whenever possible. Additionally, watching videos with authentic content can be helpful, with FluentU being a platform highly recommended by Eleo for expanding vocabulary and practicing listening skills.
Thought:Do I need to use a tool? No
AI: Eleo recommends practicing German pronunciation by speaking to oneself in the language being learned during daily activities. It is also suggested to engage in conversations with native speakers or other German speakers whenever possible. Additionally, watching videos with authentic content can be helpful, with FluentU being a platform highly recommended by